In [264]:
import numpy as np

In [265]:
maze = [[0, 1, 0, -1, 0],
        [0, 0, 0, 0, 0],
        [-1, 0, 0, 1, -1],
        [1, 0, 0, 0, 3],
        [0, 0, -1, 0, -1]]
# -1 is bomb site, 3 is final goal, 1 is wall
maze = np.array(maze)

In [266]:
value_table = np.zeros((maze.shape[0], maze.shape[1]))

In [267]:
value_table = np.zeros((maze.shape[0], maze.shape[1]))
for i in range(maze.shape[0]):
    for j in range(maze.shape[1]):
        if maze[i][j] == 1:
            value_table[i][j] = -100
        else:
            continue

In [268]:
reward_table = np.zeros((maze.shape[0], maze.shape[1]))
for i in range(maze.shape[0]):
    for j in range(maze.shape[1]):
        if maze[i][j] == 1:
            reward_table[i][j] = -100
        elif maze[i][j] == 3:
            reward_table[i][j] = 1
        elif maze[i][j] == -1:
            reward_table[i][j] = -1
        else:
            continue

In [269]:
reward_table

array([[   0., -100.,    0.,   -1.,    0.],
       [   0.,    0.,    0.,    0.,    0.],
       [  -1.,    0.,    0., -100.,   -1.],
       [-100.,    0.,    0.,    0.,    1.],
       [   0.,    0.,   -1.,    0.,   -1.]])

In [270]:
maze.shape[0], maze.shape[1]

(5, 5)

In [271]:
def is_valid_state(state, maze):
    if state[0] >= maze.shape[0] or state[0] < 0 or state[1] >= maze.shape[1] or state[1] < 0:
        return False
    if maze[state[0]][state[1]] == 1:
        return False
    else:
        return True

In [272]:
def get_max_value(state, maze, reward_table, value_table, discount):
    if not is_valid_state(state, maze):
        return False
    else:
        values = []
        if is_valid_state((state[0] + 1, state[1]), maze):
            v = value_table[state[0] + 1][state[1]] * discount + reward_table[state[0]][state[1]]
            values.append(v)
        if is_valid_state((state[0], state[1] + 1), maze):
            v = value_table[state[0]][state[1] + 1] * discount + reward_table[state[0]][state[1]]
            values.append(v)
        if is_valid_state((state[0] - 1, state[1]), maze):
            v = value_table[state[0] - 1][state[1]] * discount + reward_table[state[0]][state[1]]
            values.append(v)
        if is_valid_state((state[0], state[1] - 1), maze):
            v = value_table[state[0]][state[1] - 1] * discount + reward_table[state[0]][state[1]]
            values.append(v)
        max_val = max(values)
        value_table[state[0]][state[1]] = max_val
    return True
        

In [273]:
value_table

array([[   0., -100.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0., -100.,    0.],
       [-100.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.]])

In [274]:
reward_table

array([[   0., -100.,    0.,   -1.,    0.],
       [   0.,    0.,    0.,    0.,    0.],
       [  -1.,    0.,    0., -100.,   -1.],
       [-100.,    0.,    0.,    0.,    1.],
       [   0.,    0.,   -1.,    0.,   -1.]])

In [275]:
def value_table_update(maze, reward_table, value_table, discount = 0.95):
    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            state = (i, j)
            update = get_max_value(state, maze, reward_table, value_table, discount)

In [276]:
for i in range(100):
    value_table_update(maze, reward_table, value_table, discount = 0.95)
print(value_table)

[[   7.16191899 -100.            7.93575012    6.53896262    7.89067042]
 [   7.53891493    7.93575012    8.35346887    7.93579542    8.3060119 ]
 [   6.93575012    8.35346887    8.79317042 -100.            8.74321131]
 [-100.            8.79317042    9.2560119     9.74321131   10.25605074]
 [   7.93579542    8.3535119     7.79321131    9.25605074    8.7432482 ]]


In [277]:
def get_best_state(state, maze, value_table):
    x, y = state
    possible_states = [(x + 1, y), (x, y + 1), (x - 1, y), (x, y - 1)]
    values = []
    for s in possible_states:
        if not is_valid_state(s, maze):
            values.append(-100)
        else:
            values.append(value_table[*s])
    index = np.array(values).argmax()
    return possible_states[index]
def solve_maze(starting_state, maze, value_table):
    value = 0
    curr_state = starting_state
    final_state = np.where(maze == 3)
    while not curr_state == final_state:
        if not is_valid_state(curr_state, maze):
            print("INVALID STATE REACHED\nSTATE : ", curr_state)
            return False
        value += value_table[*curr_state]
        maze[*curr_state] = +100
        # value[*curr_state] = -1000
        curr_state = get_best_state(curr_state, maze, value_table)
    return True

In [278]:
solve_maze((0, 0), maze, value_table)

True

In [279]:
maze

array([[100,   1,   0,  -1,   0],
       [100, 100,   0,   0,   0],
       [ -1, 100,   0,   1,  -1],
       [  1, 100, 100, 100,   3],
       [  0,   0,  -1,   0,  -1]])

In [280]:
value_table

array([[   7.16191899, -100.        ,    7.93575012,    6.53896262,
           7.89067042],
       [   7.53891493,    7.93575012,    8.35346887,    7.93579542,
           8.3060119 ],
       [   6.93575012,    8.35346887,    8.79317042, -100.        ,
           8.74321131],
       [-100.        ,    8.79317042,    9.2560119 ,    9.74321131,
          10.25605074],
       [   7.93579542,    8.3535119 ,    7.79321131,    9.25605074,
           8.7432482 ]])